In [ ]:
!pip install feature_engine
!pip install tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 326.6/326.6 kB 6.0 MB/s eta 0:00:00


In [ ]:
# import library

import pandas as pd
import numpy as np
import joblib
import sklearn.preprocessing

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# load models
with open('/../pipeline.pkl','rb') as file_pkl:
  model_pipeline = joblib.load(file_pkl)

from tensorflow.keras.models import load_model
model = load_model('/../churn_model.h5')

In [ ]:
# Make a new testing dataset -> for inference

df_new = pd.DataFrame({
    'membership_category':['Basic Membership','Silver Membership','Platinum Membership'],
    'avg_transaction_value':[8371.23, 42903.90, 780922.11],
    'points_in_wallet':[456.97000,789.93000,800.49000],
    'feedback':['Poor Customer Service','Too many ads','Products always in Stock']
})

df_new

,membership_category,avg_transaction_value,points_in_wallet,feedback
0,Basic Membership,8371.23,456.97,Poor Customer Service
1,Silver Membership,42903.90,789.93,Too many ads
2,Platinum Membership,780922.11,800.49,Products always in Stock


In [ ]:
# Apply the pipeline to dataset
df_new_transform = model_pipeline.transform(df_new)
df_new_transform

array([[0.07638478, 0.33481309, 0.        , 1.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 1.        ],
       [0.42479987, 0.59619189, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 1.        ,
        0.        , 2.        ],
       [1.20784869, 0.60448165, 0.        , 0.        , 0.        ,
        0.        , 1.        , 0.        , 0.        , 0.        ,
        0.        , 5.        ]])

In [ ]:
# Make predictions on dataset

y_pred = model.predict(df_new_transform)
y_pred = np.where(y_pred >= 0.5, 1, 0)
y_pred

1/1 [==============================] - 3s 3s/step


array([[1],
       [0],
       [0]])

Concat the prediction results with the dataframe

In [ ]:
y_pred_df = pd.DataFrame(y_pred, columns=['churn_risk_score_pred'])
df_concat = pd.concat([df_new, y_pred_df], axis=1)
df_concat

,membership_category,avg_transaction_value,points_in_wallet,feedback,churn_risk_score_pred
0,Basic Membership,8371.23,456.97,Poor Customer Service,1
1,Silver Membership,42903.90,789.93,Too many ads,0
2,Platinum Membership,780922.11,800.49,Products always in Stock,0
